# Анализ бизнес показателей

Несмотря на огромные вложения в рекламу, последние несколько месяцев развлекательное приложение Procrastinate Pro+ терпит убытки. 

**Задачи**: 

- определить причины неэффективного привлечения пользователей;
- сформулируйте рекомендации для отдела маркетинга. 

**План исследования**:

- изучить полученные данные о визитах, заказах и рекламных расходах;
- выполнить предобработку данных;
- задать функции для определения основных метрик (LTV, ROI, conversion, retention rate) и построения соответствующих графиков;
- составить профили пользователей;
- изучить, из каких стран пользователи приходят в приложение, и определить долю платящих в каждой из них;
- определить, какими устройствами пользуются клиенты, и какие устройства предпочитают платящие пользователи;
- изучить рекламные источники привлечения и определить каналы, из которых пришло больше всего платящих пользователей. 
- выяснить, как траты распределены по рекламным источникам;
- построить визуализацию динамики изменения расходов во времени (по неделям и месяцам) по каждому источнику;
- узнать, сколько в среднем стоило привлечение одного пользователя (CAC) из каждого источника;
- проанализировать окупаемость рекламы c помощью графиков LTV и ROI, а также графиков динамики LTV, CAC и ROI;
- проверить конверсию пользователей и динамику её изменения;
- проанализировать окупаемость рекламы с разбивкой по устройствам, странам, рекламным каналам;
- сформулировать вывод и рекомендации.

**Описание данных**:

В наличии имеются данные о пользователях, привлечённых с 1 мая по 27 октября 2019 года:
- лог сервера с данными об их посещениях,
- выгрузка их покупок за этот период,
- рекламные расходы.


### Подготвка данных к анализу

#### Импорт библиотек и выгрузка данных.

In [1]:
# Импортируем библиотеки

import pandas as pd
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns


In [2]:
# Загружаем данные 

visits = pd.read_csv('/datasets/visits_info_short.csv')
orders = pd.read_csv('/datasets/orders_info_short.csv')
costs = pd.read_csv('/datasets/costs_info_short.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/visits_info_short.csv'

#### Общее знакомство с данными.

In [ ]:
visits.head()

In [ ]:
visits.info()

In [ ]:
visits.isna().sum()

In [ ]:
visits.duplicated().sum()

В таблице visits 6 столбцов: User Id, Region, Device, Channel, Session Start, Session End. Количество имеющихся набюдений - 309901 в каждом столбце. Пропусков в данных нет. Полные дубликаты отсутствуют.

In [ ]:
orders.head()


In [ ]:
orders.info()

In [ ]:
orders.isna().sum()

In [ ]:
orders.duplicated().sum()

В таблице orders с информацией о заказах 3 столбца: User Id, Event Dt, Revenue. Количество имеющихся наблюдений -  40212 в каждом столбце.  Пропусков в данных нет. Полные дубликаты отсутствуют.

In [ ]:
costs.head()

In [ ]:
costs.info()

In [ ]:
costs.isna().sum()

In [ ]:
costs.duplicated().sum()

В таблице costs с данными о затратах на рекламу 3 столбца: dt, Channel  и costs. Количество имеющихся наблюдений в каждом - 1800. Пропусков в данных нет. Полные дубликаты отсутствуют.   

#### Изменение названий столбцов.

In [ ]:
# Приведём названия столбцов в данных к нижнему регистру.

visits.columns = visits.columns.str.lower()

In [ ]:
visits = visits.rename(columns = {'user id':'user_id', 'session start':'session_start', 'session end':'session_end' })
visits.head()

In [ ]:
orders.columns = orders.columns.str.lower()

In [ ]:
orders = orders.rename(columns = {'user id':'user_id', 'event dt':'event_dt'})
orders.head()

In [ ]:
costs = costs.rename(columns = {'Channel':'channel'})
costs.head()

#### Изменение типов данных.

In [ ]:
# преобразуем данные о времени в таблице visits для дальнейших расчётов

visits['session_start'] = pd.to_datetime(visits['session_start'])


In [ ]:
visits['session_end'] = pd.to_datetime(visits['session_end'])

In [ ]:
# преобразуем данные о времени в таблице orders 

orders['event_dt'] = pd.to_datetime(orders['event_dt'])

In [ ]:
# преобразуем данные о времени в таблице costs

costs['dt'] = pd.to_datetime(costs['dt']).dt.date

*Краткие выводы:*

В процессе подготовки данных к анализу были загружены данные с информацией о визитах, покупках и тратах на рекламу. Каждая из таблиц была проверена на наличие пропусков и дубликатов. Во всех имеющихся данных пропуски и дубликаты не обнаружены. Для удобства в последующей работе, названия столбцов были приведены к нижнему регистру, а также был изменён тип данных в столбцах с информацией о времени: в таблице о визитах - столбцы с временем начала и окончани сессий,  в таблице с покупками - с временем совершения покупки, в таблице с затратами на рекламу - с датой рекламной кампании.

### Задаём функции для расчёта и анализа LTV, ROI, удержания и конверсии.

#### Функция для создания пользовательских профилей

In [ ]:
# функция для создания пользовательских профилей

def get_profiles(sessions, orders, ad_costs):

    # находим параметры первых посещений
    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()
    )

    # для когортного анализа определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    # добавляем признак платящих пользователей
    profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique())


    # считаем количество уникальных пользователей
    # с одинаковыми источником и датой привлечения
    new_users = (
        profiles.groupby(['dt', 'channel'])
        .agg({'user_id': 'nunique'})
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()
    )

    # объединяем траты на рекламу и число привлечённых пользователей
    ad_costs = ad_costs.merge(new_users, on=['dt', 'channel'], how='left')

    # делим рекламные расходы на число привлечённых пользователей
    ad_costs['acquisition_cost'] = ad_costs['costs'] / ad_costs['unique_users']

    # добавляем стоимость привлечения в профили
    profiles = profiles.merge(
        ad_costs[['dt', 'channel', 'acquisition_cost']],
        on=['dt', 'channel'],
        how='left',
    )

    # стоимость привлечения органических пользователей равна нулю
    profiles['acquisition_cost'] = profiles['acquisition_cost'].fillna(0)

    return profiles


#### Функция для расчёта удержания

In [ ]:
# функция для расчёта удержания

def get_retention(
    profiles,
    sessions,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # добавляем столбец payer в передаваемый dimensions список
    dimensions = ['payer'] + dimensions

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # собираем «сырые» данные для расчёта удержания
    result_raw = result_raw.merge(
        sessions[['user_id', 'session_start']], on='user_id', how='left'
    )
    result_raw['lifetime'] = (
        result_raw['session_start'] - result_raw['first_ts']
    ).dt.days

    # функция для группировки таблицы по желаемым признакам
    
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу удержания
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # получаем таблицу динамики удержания
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time


#### Функция для расчёта конверсии

In [ ]:
# функция для расчёта конверсии

def get_conversion(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # определяем дату и время первой покупки для каждого пользователя
    first_purchases = (
        purchases.sort_values(by=['user_id', 'event_dt'])
        .groupby('user_id')
        .agg({'event_dt': 'first'})
        .reset_index()
    )

    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        first_purchases[['user_id', 'event_dt']], on='user_id', how='left'
    )

    # рассчитываем лайфтайм для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days

    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users' 
        dimensions = dimensions + ['cohort']

    # функция для группировки таблицы по желаемым признакам
    
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        result = result.fillna(0).cumsum(axis = 1)
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # делим каждую «ячейку» в строке на размер когорты
        # и получаем conversion rate
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу конверсии
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # для таблицы динамики конверсии убираем 'cohort' из dimensions
    if 'cohort' in dimensions: 
        dimensions = []

    # получаем таблицу динамики конверсии
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time 


#### Функция для расчёта LTV и ROI

In [ ]:
# функция для расчёта LTV и ROI

def get_ltv(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')
    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        purchases[['user_id', 'event_dt', 'revenue']], on='user_id', how='left'
    )
    # рассчитываем лайфтайм пользователя для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days
    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # функция группировки по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        # строим «треугольную» таблицу выручки
        result = df.pivot_table(
            index=dims, columns='lifetime', values='revenue', aggfunc='sum'
        )
        # находим сумму выручки с накоплением
        result = result.fillna(0).cumsum(axis=1)
        # вычисляем размеры когорт
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        # объединяем размеры когорт и таблицу выручки
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # считаем LTV: делим каждую «ячейку» в строке на размер когорты
        result = result.div(result['cohort_size'], axis=0)
        # исключаем все лайфтаймы, превышающие горизонт анализа
        result = result[['cohort_size'] + list(range(horizon_days))]
        # восстанавливаем размеры когорт
        result['cohort_size'] = cohort_sizes

        # собираем датафрейм с данными пользователей и значениями CAC, 
        # добавляя параметры из dimensions
        cac = df[['user_id', 'acquisition_cost'] + dims].drop_duplicates()

        # считаем средний CAC по параметрам из dimensions
        cac = (
            cac.groupby(dims)
            .agg({'acquisition_cost': 'mean'})
            .rename(columns={'acquisition_cost': 'cac'})
        )

        # считаем ROI: делим LTV на CAC
        roi = result.div(cac['cac'], axis=0)

        # удаляем строки с бесконечным ROI
        roi = roi[~roi['cohort_size'].isin([np.inf])]

        # восстанавливаем размеры когорт в таблице ROI
        roi['cohort_size'] = cohort_sizes

        # добавляем CAC в таблицу ROI
        roi['cac'] = cac['cac']

        # в финальной таблице оставляем размеры когорт, CAC
        # и ROI в лайфтаймы, не превышающие горизонт анализа
        roi = roi[['cohort_size', 'cac'] + list(range(horizon_days))]

        # возвращаем таблицы LTV и ROI
        return result, roi

    # получаем таблицы LTV и ROI
    result_grouped, roi_grouped = group_by_dimensions(
        result_raw, dimensions, horizon_days
    )

    # для таблиц динамики убираем 'cohort' из dimensions
    if 'cohort' in dimensions:
        dimensions = []

    # получаем таблицы динамики LTV и ROI
    result_in_time, roi_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    return (
        result_raw,  # сырые данные
        result_grouped,  # таблица LTV
        result_in_time,  # таблица динамики LTV
        roi_grouped,  # таблица ROI
        roi_in_time,  # таблица динамики ROI
    )


#### Функция для сглаживания фрейма

In [ ]:
# функция для сглаживания фрейма

def filter_data(df, window):
    # для каждого столбца применяем скользящее среднее
    for column in df.columns.values:
        df[column] = df[column].rolling(window).mean() 
    return df 


#### Функция для визуализации удержания

In [ ]:
# функция для визуализации удержания

def plot_retention(retention, retention_history, horizon, window=7):

    # задаём размер сетки для графиков
    plt.figure(figsize=(15, 10))

    # исключаем размеры когорт и удержание первого дня
    retention = retention.drop(columns=['cohort_size', 0])
    # в таблице динамики оставляем только нужный лайфтайм
    retention_history = retention_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # если в индексах таблицы удержания только payer,
    # добавляем второй признак — cohort
    if retention.index.nlevels == 1:
        retention['cohort'] = 'All users'
        retention = retention.reset_index().set_index(['cohort', 'payer'])

    # в таблице графиков — два столбца и две строки, четыре ячейки
    # в первой строим кривые удержания платящих пользователей
    ax1 = plt.subplot(2, 2, 1)
    retention.query('payer == True').droplevel('payer').T.plot(
        grid=True, ax=ax1
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание платящих пользователей')

    # во второй ячейке строим кривые удержания неплатящих
    # вертикальная ось — от графика из первой ячейки
    ax2 = plt.subplot(2, 2, 2, sharey=ax1)
    retention.query('payer == False').droplevel('payer').T.plot(
        grid=True, ax=ax2
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание неплатящих пользователей')

    # в третьей ячейке — динамика удержания платящих
    ax3 = plt.subplot(2, 2, 3)
    # получаем названия столбцов для сводной таблицы
    columns = [
        name
        for name in retention_history.index.names
        if name not in ['dt', 'payer']
    ]
    # фильтруем данные и строим график
    filtered_data = retention_history.query('payer == True').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания платящих пользователей на {}-й день'.format(
            horizon
        )
    )

    # в чётвертой ячейке — динамика удержания неплатящих
    ax4 = plt.subplot(2, 2, 4, sharey=ax3)
    # фильтруем данные и строим график
    filtered_data = retention_history.query('payer == False').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax4)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания неплатящих пользователей на {}-й день'.format(
            horizon
        )
    )
    
    plt.tight_layout()
    plt.show() 



#### Функция для визуализации конверсии

In [ ]:
# функция для визуализации конверсии

def plot_conversion(conversion, conversion_history, horizon, window=7):

    # задаём размер сетки для графиков
    plt.figure(figsize=(15, 5))

    # исключаем размеры когорт
    conversion = conversion.drop(columns=['cohort_size'])
    # в таблице динамики оставляем только нужный лайфтайм
    conversion_history = conversion_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # первый график — кривые конверсии
    ax1 = plt.subplot(1, 2, 1)
    conversion.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Конверсия пользователей')

    # второй график — динамика конверсии
    ax2 = plt.subplot(1, 2, 2, sharey=ax1)
    columns = [
        # столбцами сводной таблицы станут все столбцы индекса, кроме даты
        name for name in conversion_history.index.names if name not in ['dt']
    ]
    filtered_data = conversion_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика конверсии пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show()



#### Функция для визуализации LTV и ROI

In [ ]:
# функция для визуализации LTV и ROI

def plot_ltv_roi(ltv, ltv_history, roi, roi_history, horizon, window=7):

    # задаём сетку отрисовки графиков
    plt.figure(figsize=(20, 10))

    # из таблицы ltv исключаем размеры когорт
    ltv = ltv.drop(columns=['cohort_size'])
    # в таблице динамики ltv оставляем только нужный лайфтайм
    ltv_history = ltv_history.drop(columns=['cohort_size'])[[horizon - 1]]

    # стоимость привлечения запишем в отдельный фрейм
    cac_history = roi_history[['cac']]

    # из таблицы roi исключаем размеры когорт и cac
    roi = roi.drop(columns=['cohort_size', 'cac'])
    # в таблице динамики roi оставляем только нужный лайфтайм
    roi_history = roi_history.drop(columns=['cohort_size', 'cac'])[
        [horizon - 1]
    ]

    # первый график — кривые ltv
    ax1 = plt.subplot(2, 3, 1)
    ltv.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('LTV')

    # второй график — динамика ltv
    ax2 = plt.subplot(2, 3, 2, sharey=ax1)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in ltv_history.index.names if name not in ['dt']]
    filtered_data = ltv_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика LTV пользователей на {}-й день'.format(horizon))

    # третий график — динамика cac
    ax3 = plt.subplot(2, 3, 3, sharey=ax1)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in cac_history.index.names if name not in ['dt']]
    filtered_data = cac_history.pivot_table(
        index='dt', columns=columns, values='cac', aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика стоимости привлечения пользователей')

    # четвёртый график — кривые roi
    ax4 = plt.subplot(2, 3, 4)
    roi.T.plot(grid=True, ax=ax4)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('ROI')

    # пятый график — динамика roi
    ax5 = plt.subplot(2, 3, 5, sharey=ax4)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in roi_history.index.names if name not in ['dt']]
    filtered_data = roi_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax5)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.xlabel('Дата привлечения')
    plt.title('Динамика ROI пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show()


<div class="alert alert-success">
  
  <b>Комментарий ревьюера</b>
    
  ✔️ Отличные функции!
</div>

### Исследовательский анализ данных

- Составьте профили пользователей. Определите минимальную и максимальную даты привлечения пользователей.
- Выясните, из каких стран пользователи приходят в приложение и на какую страну приходится больше всего платящих пользователей. Постройте таблицу, отражающую количество пользователей и долю платящих из каждой страны.
- Узнайте, какими устройствами пользуются клиенты и какие устройства предпочитают платящие пользователи. Постройте таблицу, отражающую количество пользователей и долю платящих для каждого устройства.
- Изучите рекламные источники привлечения и определите каналы, из которых пришло больше всего платящих пользователей. Постройте таблицу, отражающую количество пользователей и долю платящих для каждого канала привлечения.

После каждого пункта сформулируйте выводы.

#### Составление профилей

In [ ]:
# Составляем профили пользователей

profiles = get_profiles(visits, orders, costs)
profiles.head()

<div class="alert alert-warning">
    
  <b>Комментарий ревьюера</b>
    
  ⚠️ Не выводи на показ весь датафрейм, при больших объемах можно подвесить ноутбук. Используй head(), tail() или sample().
</div>

<div class="alert alert-block alert-info">

  <b>Комментарий cтудента</b>

  Вывела только первые 5 строк выше.
</div>
<div class="alert alert-success">
  
  <b>Комментарий ревьюера v.2</b>
    
  ✔️ 👌
</div>

In [ ]:
# Определяем минимальную дату привлечения пользователей

min_analysis_date = profiles['dt'].min()
min_analysis_date

In [ ]:
# Определяем максимальную дату привлечения пользователей

max_analysis_date = profiles['dt'].max()
max_analysis_date

После составления профилей пользователей в нашем распоряжении оказалось 150008 наблюдений, в каждом из которых есть данные о дате первого посещения, рекламном источнике, используемом устройстве, стране и стоимости привлечения пользователя, а также признак payer (совершил ли пользователь покупку). Минимальная дата привлечения пользователей в имеющихся данных - 1 мая 2019г., а максимальная - 27 октября 2019г., что полностью совпадает с полученной в задании информацией. 

<div class="alert alert-success">
  
  <b>Комментарий ревьюера</b>
    
  ✔️ Замечательно, у нас есть даты!
</div>
<div class="alert alert-warning">
    
  <b>Комментарий ревьюера</b>
    
  ⚠️ Стоит сказать, что они совпадают с заданием.
</div>

<div class="alert alert-block alert-info">

  <b>Комментарий cтудента</b>

  Дополнила этот пункт к выводу.
</div>
<div class="alert alert-success">
  
  <b>Комментарий ревьюера v.2</b>
    
  ✔️ 👌
</div>

#### Количество пользователей по странам, доля платящих

In [ ]:
# Определяем, из каких стран приходят пользователи и их количество
# определяем долю платящих в каждой стране

users_region = profiles.pivot_table(index='region', values=['user_id', 'payer'], aggfunc = {'payer':['mean', 'sum'], 'user_id':'nunique'})
users_region.columns = ['payers_share', 'payers_number', 'users_number'] 
users_region['payers_share'] = round(users_region['payers_share']*100, 2)
users_region.sort_values(by = 'payers_share', ascending = False)                        


Пользователи приложения приходят из США, Германии, Великобритании и Франции. Количество новых пользователей из США значительно превышает количество пользователей из других стран - 100002. Из Великобритении и Франции пришло в приложение примерно одинаковое количество пользователей - 17575 и 17450 соответственно. Меньше всего пользователей пришло из Германии - 14981. Самая высокая доля платящих пользователей из США - 6.9%. Далее идут Германия (4.11%), Великобритания (3.98%) и Франция(3.8%).

#### Количество пользователей по типу устройста, доля платящих

In [ ]:
# Определяем, какими устройствами пользуются клиенты 
# и какие устройства предпочитают платящие пользователи

users_device = profiles.pivot_table(index='device', values=['user_id', 'payer'], aggfunc = {'payer':['mean', 'sum'], 'user_id':'nunique'})
users_device.columns = ['payers_share', 'payers_number', 'users_number'] 
users_device['payers_share'] = round(users_device['payers_share']*100, 2)
users_device.sort_values(by = 'payers_share', ascending = False)


Большая часть пользователей для входа в приложение использует iPhone - 54479. Второе по популярности устройство - Android (35032). Примерно одинаковое количество пользователей использует для входа в приложение PC и Mac (30455 и 30042 соответственно). Наиболее высокие доли платящих у пользователей Apple: Mac - 6.4%, iPhone - 6.2%. Несколько ниже доля платящих у пользователей Android (5.9%). Самый низкий показатель у пользователей PC - 5%.

#### Количество пользователей по источнику привлечения, доля платящих.

In [ ]:
# Определяем источники привлечения и каналы, из которых пришло больше всего платящих пользователей

users_channel = profiles.pivot_table(index='channel', values=['user_id', 'payer'], aggfunc = {'payer':['mean', 'sum'], 'user_id':'nunique'})
users_channel.columns = ['payers_share', 'payers_number', 'users_number'] 
users_channel['payers_share'] = round(users_channel['payers_share']*100, 2)
users_channel.sort_values(by = 'payers_share', ascending = False)

Хотя количество органических пользователей самое значительное, доля платящих среди них самая низкая (2.1%). Самая высокая доля платящих приходит через FaceBoom (12.2%). Также высокая доля платящих приходит через AdNonSense (11.34%), lambdaMediaAds (10.47%) и TipTop (9.6%).

<div class="alert alert-success">
  
  <b>Комментарий ревьюера</b>
    
  ✔️ С платящими определились!
</div>

*Выводы*:

* После составления профилей пользователей в нашем распоряжении оказалось 150008 наблюдений, в каждом из которых есть данные о дате первого посещения, рекламном источнике, используемом устройстве, стране и стоимости привлечения пользователя, а также признак payer (совершил ли пользователь покупку). Минимальная дата привлечения пользователей в имеющихся данных - 1 мая 2019г., а максимальная - 27 октября 2019г., что полностью совпадает с полученной в задании информацией. 
* Пользователи приложения приходят из США, Германии, Великобритании и Франции. Количество новых пользователей из США значительно превышает количество пользователей из других стран - 100002. Из Великобритении и Франции пришло в приложение примерно одинаковое количество пользователей - 17575 и 17450 соответственно. Меньше всего пользователей пришло из Германии - 14981. Самая высокая доля платящих пользователей из США - 6.9%. Далее идут Германия (4.11%), Великобритания (3.98%) и Франция(3.8%).
* Большая часть пользователей для входа в приложение использует iPhone - 54479. Второе по популярности устройство - Android (35032). Примерно одинаковое количество пользователей использует для входа в приложение PC и Mac (30455 и 30042 соответственно). Наиболее высокие доли платящих у пользователей Apple: Mac - 6.4%, iPhone - 6.2%. Несколько ниже доля платящих у пользователей Android (5.9%). Самый низкий показатель у пользователей PC - 5%.
* Хотя количество органических пользователей самое значительное, доля платящих среди них самая низкая (2.1%). Самая высокая доля платящих приходит через FaceBoom (12.2%). Также высокая доля платящих приходит через AdNonSense (11.34%), lambdaMediaAds (10.47%) и TipTop (9.6%).

### Маркетинг

- Посчитайте общую сумму расходов на маркетинг.
- Выясните, как траты распределены по рекламным источникам, то есть сколько денег потратили на каждый источник.
- Постройте визуализацию динамики изменения расходов во времени (по неделям и месяцам) по каждому источнику. Постарайтесь отразить это на одном графике.
- Узнайте, сколько в среднем стоило привлечение одного пользователя (CAC) из каждого источника. Используйте профили пользователей.

Напишите промежуточные выводы.

#### Общая сумма расходов на маркетинг

In [ ]:
# Определяем общую сумму расходов на маркетинг 

costs_total = costs['costs'].sum()
round(costs_total)

Всего за исследуемы период на рекламу было потрачено 105497 

<div class="alert alert-success">
  
  <b>Комментарий ревьюера</b>
    
  ✔️ Общие затраты посчитали.
</div>

#### Траты по источникам рекламы

In [ ]:
# Определим, как распределились траты по рекламным источникам.

channel_costs = costs.pivot_table(index='channel', values = 'costs', aggfunc = 'sum')
channel_costs['share'] = round(channel_costs['costs']/costs_total*100, 2)
channel_costs.sort_values('costs', ascending=False)

Больше всего средств было вложено на рекламу в TipTop (54751.3) и FaceBoom (32445.6). При этом больше половины рекламного бюджета было выделено на рекламу на TipTop. 

<div class="alert alert-success">
  
  <b>Комментарий ревьюера</b>
    
  ✔️  На популярные каналы тратится больше.

</div> 
<div class="alert alert-warning">
    
  <b>Комментарий ревьюера</b>
      
  ⚠️ Стоит вывести еще процент затрат каждого канала от общей суммы, так будет понятней общая картина.
</div>

<div class="alert alert-block alert-info">

  <b>Комментарий cтудента</b>

  Добавила столбец с процентом затрат.
</div>
<div class="alert alert-success">
  
  <b>Комментарий ревьюера v.2</b>
    
  ✔️ Замечательно! Видим, что два канала съели более 80% бюджета.
</div>

#### Динамика изменения расходов во времени (по неделям и месяцам) по каждому источнику.

In [ ]:
# Создаем в таблице costs столбцы с номером месяца и недели

costs['week'] = pd.to_datetime(costs['dt']).dt.isocalendar().week
costs['month'] = pd.DatetimeIndex(costs['dt']).month
costs.head()

In [ ]:
# Строим графики динамики изменения расходов во времени (по неделям и месяцам) по каждому источнику

plt.figure(figsize=(20, 8))

cac_week = costs.pivot_table(index='week', columns='channel', values='costs', aggfunc='sum')\
                .plot(grid=True, xticks=list(costs.week.values), ax = plt.subplot(1, 2, 1))
plt.ylabel('Сумма расходов', fontsize = 15)
plt.xlabel('Номер недели', fontsize = 15)
plt.title('Динамика САС по рекламным источникам', fontsize = 20)
plt.legend(fontsize = 13)


cac_month = costs.pivot_table(index='month', columns='channel', values='costs', aggfunc='sum')\
                 .plot(grid=True, ax = plt.subplot(1, 2, 2))
plt.ylabel('Сумма расходов', fontsize = 15)
plt.xlabel('Номер месяца', fontsize = 15)
plt.title('Динамика САС по рекламным источникам', fontsize = 20)
plt.legend(fontsize = 13)
plt.tight_layout()
plt.show()

Графики показывают, что больше всего средств было потрачено на рекламу в TipTop и Faceboom. При этом в мае траты на рекламу в Faceboom были несколько выше, но примерно с 21 недели (с июня) TipTop вышел в лидеры. Вклад в рекламу на этом ресурсе стабильно рос до 9 месяца с заметным пиком на 39 неделе.

<div class="alert alert-success">
  
  <b>Комментарий ревьюера</b>
    
  ✔️ Видим динамику расходов. TipTop и FaceBoom заметно выделяются.
</div>

#### Средняя стоимость привлечения пользователя по рекламным источникам.

In [ ]:
cac_mean = profiles.pivot_table(index='channel', values = 'acquisition_cost', aggfunc = 'mean')
cac_mean['acquisition_cost'] = round(cac_mean['acquisition_cost'], 2)
cac_mean.sort_values('acquisition_cost', ascending = False)

Самыми дорогими по привлечению пользователями оказались пользователи, пришедшие через TipTop (2.8), FaceBoom (1.11) и AdNonSense (1.01).	

<div class="alert alert-success">
  
  <b>Комментарий ревьюера</b>
    
  ✔️ И обходятся популярные каналы дороже.

</div> 


<div class="alert alert-warning">
    
  <b>Комментарий ревьюера</b>
    
  ⚠️ Можно еще рассчитать средний CAC на одного пользователя для всего проекта, так увидим на сколько выше среднего САС у популярных каналов.
    
</div>
<div class="alert alert-danger">
    
  <b>Комментарий ревьюера</b>
    
  ❌ Напрашивается промежуточный вывод по всему шагу маркетинга.
</div>

<div class="alert alert-block alert-info">

  <b>Комментарий cтудента</b>

  Сергей, по твоей рекомендации решила рассчитать средний CAC. Я же правильно понимаю, что речь идёт о неорганических пользователях? Органические же для нас бесплатны.
    
</div>
<div class="alert alert-success">
  
  <b>Комментарий ревьюера v.2</b>
    
  ✔️ Верно! CAC (Customer Acquisition Cost - стоимость привлечения клиента) это сумма, в которую нам обходится каждый новый клиент совершивший покупку. А за клиентов канала organic мы не платим.
</div>

In [ ]:
# Рассчитаем средний CAC на одного пользователя для всего проекта.
# Найдём общее количество платящих пользователей.

profiles_filt = profiles.query('channel != "organic"')

users_nonorg = profiles_filt['user_id'].count()
users_nonorg

In [ ]:
CAC_mean = round(costs_total/users_nonorg, 2)
CAC_mean

Если сравнить траты по каналам  со средним CAC по проекту, то видно, что рекламные расходы на TipTop более чем в 2 раза превышают средний показатель.

**Выводы:**
* Всего за исследуемы период на рекламу было потрачено 105497.
* Больше всего средств было вложено на рекламу в TipTop (54751.3) и FaceBoom (32445.6). При этом больше половины рекламного бюджета было выделено на рекламу на TipTop. При этом в мае траты на рекламу в Faceboom были несколько выше, но примерно с 21 недели (с июня) TipTop вышел в лидеры. Вклад в рекламу на этом ресурсе стабильно рос до 9 месяца с заметным пиком на 39 неделе.
* Самыми дорогими по привлечению пользователями оказались пользователи, пришедшие через TipTop (2.8), FaceBoom (1.11) и AdNonSense (1.01).

### Оцените окупаемость рекламы

Используя графики LTV, ROI и CAC, проанализируйте окупаемость рекламы. Считайте, что на календаре 1 ноября 2019 года, а в бизнес-плане заложено, что пользователи должны окупаться не позднее чем через две недели после привлечения. Необходимость включения в анализ органических пользователей определите самостоятельно.

- Проанализируйте окупаемость рекламы c помощью графиков LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проверьте конверсию пользователей и динамику её изменения. То же самое сделайте с удержанием пользователей. Постройте и изучите графики конверсии и удержания.
- Проанализируйте окупаемость рекламы с разбивкой по устройствам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проанализируйте окупаемость рекламы с разбивкой по странам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проанализируйте окупаемость рекламы с разбивкой по рекламным каналам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Ответьте на такие вопросы:
    - Окупается ли реклама, направленная на привлечение пользователей в целом?
    - Какие устройства, страны и рекламные каналы могут оказывать негативное влияние на окупаемость рекламы?
    - Чем могут быть вызваны проблемы окупаемости?

Напишите вывод, опишите возможные причины обнаруженных проблем и промежуточные рекомендации для рекламного отдела.

#### LTV, CAC и ROI

In [ ]:
# Графики LTV и ROI, а также графики динамики LTV, CAC и ROI

horizon_days=14
observation_date = datetime(2019, 11, 1).date()

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(profiles_filt, orders, observation_date, horizon_days)

plot_ltv_roi(ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days) 


**Выводы** 

- Реклама в целом не окупается. ROI к концу второй недели достигает примерно 80%.
- САС показывает значительный рост начиная с серидины мая, рекламный бюджет заметно увеличивался на протяжении всего рассматриваемого периода. В сентябре/октябре траты на рекламу примерно в 2 раза выше, чем в мае.
- В динамике LTV наблюдаются некоторые скачки и спады, но в целом показатель стабилен.
- Динамика ROI на 14 день показывает тревожные результаты. Сначала показатель опустился на некоторое время ниже линии окупаемости в начале июня, затем на короткое время незначительно перевалил за неё, а после начал показывать стабильную тенденцию к снижению.
- Хотя затраты на рекламу постоянно увеличивались, LTV оставался достаточно стабильным. Соответственно, ROI также стабильно шёл на снижение.

<div class="alert alert-success">
  
  <b>Комментарий ревьюера</b>
    
  ✔️ Да, наблюдаем явные проблемы.
</div>

#### Конверсия и удержание пользователей

In [ ]:
# Изучаем графики конверсии

conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles_filt, orders, observation_date, horizon_days)

plot_conversion(conversion_grouped, conversion_history, horizon_days)

В целом конверсия показывает постоянный рост. Наилучшие показатели конверсии на 14 день наблюдаются в середине июня.

In [ ]:
# Изучаем графики удержания

retention_raw, retention_grouped, retention_history = get_retention(
    profiles_filt, visits, observation_date, horizon_days)
    
plot_retention(retention_grouped, retention_history, horizon_days) 

Удержание платящих пользователей ожидаемо идёт на спад. Наилучшие показатели удержания наблюдаются в июне и середине августа.

#### Окупаемость рекламы с разбивкой по устройствам

In [ ]:
# Анализируем окупаемость рекламы с разбивкой по устройствам

dimensions = ['device']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(profiles_filt, orders, observation_date, horizon_days, dimensions=dimensions)

plot_ltv_roi(ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14)

In [ ]:
# смотрим конверсию с разбивкой по устройствам

dimensions = ['device']

conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles_filt, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 


In [ ]:
# смотрим удержание с разбивкой по устройствам
dimensions = ['device']

retention_raw, retention_grouped, retention_history = get_retention(
    profiles_filt, visits, observation_date, horizon_days, dimensions=dimensions
)

plot_retention(retention_grouped, retention_history, horizon_days) 


**Выводы**

- Самыми дорогими для привлечения пользователями являются пользователи iPhone и Mac, а самыми недорогими - пользователи PC.
- Между тем, окупается реклама ближе к концу второй недели только у пользователей PC. Однако, в динамике и у пользователей PC реклама перестаёт окупаться с сентября.
- Хотя конверсия пользователей iPhone и Mac самая высокая, а конверсия пользователей PC - самая низкая, удержание платящих пользователей PC стабильно самое высокое, а удержание iPhone и Mac - самое низкое.

<div class="alert alert-success">
  
  <b>Комментарий ревьюера</b>
    
  ✔️ Верно, за 14 дней окупается только PC. Но в динамике видим, что и он с сентября перестает окупаться.
</div>

#### Окупаемость рекламы с разбивкой по странам

In [ ]:
# Анализируем окупаемость рекламы с разбивкой по странам

dimensions = ['region']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(profiles_filt, orders, observation_date, horizon_days, dimensions=dimensions)

plot_ltv_roi(ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14)

In [ ]:
# смотрим конверсию с разбивкой по странам

dimensions = ['region']

conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles_filt, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 


In [ ]:
# смотрим удержание с разбивкой по странам

dimensions = ['region']

retention_raw, retention_grouped, retention_history = get_retention(
    profiles_filt, visits, observation_date, horizon_days, dimensions=dimensions
)

plot_retention(retention_grouped, retention_history, horizon_days) 


**Выводы**

- Реклама в США не окупается совсем.
- При этом стоимость привлечения пользователей в США в динамике показывает колосальный рост, начиная с середины мая, а рекламный бюджет по другим странам с конца мая наоборот снизился и остался стабильным до конца октября.
- Лучшая окупаемость рекламы в Великобритании - начиная примерно с 5 дня, а в Германии и Франции - с 6.
- В динамике LTV по всем странам достаточно стабилен.
- Хотя LTV в США самый высокий, он значительно ниже рекламных затрат.
- В динамике ROI в США показывает стабильный спад, начиная с конца мая.
- Хотя конверсия пользователей из США самая высокая (почти в 2 раза превышает конверсию в других странах), удержание платящих пользователей здесь самое низкое.


<div class="alert alert-warning">
    
  <b>Комментарий ревьюера</b>
    
  ⚠️ Согласен, можно еще изучить рекламу в США отдельно.
</div>

<div class="alert alert-block alert-info">
  
  <b>Комментарий cтудента</b>
    
  Сделала в пункте 5.6
</div>
<div class="alert alert-success">

<b>Комментарий ревьюера v.2</b>

  ✔️ Здорово! Так легче определить в США и Европе конкретные проблемы и дать рекомендации! 🙂
</div>

#### Окупаемость рекламы с разбивкой по рекламным каналам

In [ ]:
# Анализируем окупаемость рекламы с разбивкой по рекламным каналам

dimensions = ['channel']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(profiles_filt, orders, observation_date, horizon_days, dimensions=dimensions)

plot_ltv_roi(ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14)

In [ ]:
# смотрим конверсию с разбивкой по рекламным каналам

dimensions = ['channel']

conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles_filt, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 


In [ ]:
# смотрим удержание с разбивкой по рекламным каналам

dimensions = ['channel']

retention_raw, retention_grouped, retention_history = get_retention(
    profiles_filt, visits, observation_date, horizon_days, dimensions=dimensions
)

plot_retention(retention_grouped, retention_history, horizon_days) 


**Выводы**
- Реклама в TipTop, Faceboom и AdNonSense не окупается.
- Критичным образом увеличивались вложения в рекламу на TipTop, начиная с конца мая. 
- САС по другим источникам привлечения достаточно стабилен на протяжении всего периода.
- Также довольно стабилен LTV по всем каналам.
- Самая лучшая окупаемость у YRabbit.
- Хотя самый высокий рекламный бюджет у TipTop, Faceboom и AdNonSense, реклама на всех этих каналах не окупается/
- Хотя конверсия в TipTop и AdNonSense самая высокая, удержание пользователей на данных ресурсах самое низкое.


<div class="alert alert-success">
  
  <b>Комментарий ревьюера</b>
    
  ✔️ Определили проблемные каналы!
</div>

<div class="alert alert-block alert-info">
  
  <b>Комментарий cтудента</b>
    
  Сергей, ниже по твоему совету пытаюсь более детально изучить рекламу в США и Европе отдельно.
</div>

####  Реклама в США

In [ ]:
# Изучим ситуацию с рекламой отдельно в США

usa = profiles_filt.query('region == "United States"')

dimensions = ['channel']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(usa, orders, observation_date, horizon_days, dimensions=dimensions)

plot_ltv_roi(ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14)

In [ ]:
# Определим конверсию рекламных каналов в США

conversion_raw, conversion_grouped, conversion_history = get_conversion(
    usa, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_conversion(conversion_grouped, conversion_history, horizon_days)

In [ ]:
# Определим удержание рекламных каналов в США

retention_raw, retention_grouped, retention_history = get_retention(
    usa, visits, observation_date, horizon_days, dimensions=dimensions
)

plot_retention(retention_grouped, retention_history, horizon_days) 

In [ ]:
# Посмотрим, из каких каналов пришли пользователи из США

usa.groupby('channel')['user_id'].nunique().sort_values(ascending=False)

**Выводы:** получается, что большинство пользователей из США пришли в приложение через каналы, реклама в которых не окупилась - FaceBoom и TipTop. Неуспех FaceBoom связан с очень низким удержанием, а TipTop - с завышенными тратами. Необходимо скорректировать рекламный бюджет для TipTop, так как в целом на этом канале высокие и конверсия и удержание. Также стоит обратить внимание на рекламу черезе RocketSuperAds, где конверсия довольно высокая и стабильно высокое удержание.

#### Реклама в Европе

In [ ]:
# Изучим ситуацию с рекламой в Европе.

eu = profiles_filt.query('region in ["Germany", "UK", "France"]')

dimensions = ['channel']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(eu, orders, observation_date, horizon_days, dimensions=dimensions)

plot_ltv_roi(ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14)

In [ ]:
# Определим конверсию для рекламных каналов в Европе.

conversion_raw, conversion_grouped, conversion_history = get_conversion(
    eu, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_conversion(conversion_grouped, conversion_history, horizon_days) 

In [ ]:
# Определим удержание рекламных каналов в Европе

retention_raw, retention_grouped, retention_history = get_retention(
    eu, visits, observation_date, horizon_days, dimensions=dimensions
)

plot_retention(retention_grouped, retention_history, horizon_days) 

In [ ]:
# Посмотрим, из каких каналов пришли пользователи из Европы

eu.groupby('channel')['user_id'].nunique().sort_values(ascending=False)

**Выводы**: В Европе единственным каналом, который не окупается, является AdNonSense. У этого канала самый высокий САС, но самое низкое удержание. В целом за рассматриваемый период реклама в Европе окупилась за счёт того, что большинство пользователей пришли из других каналов - с высоким удержанием. Перспективным для будущих рекламных кампаний может быть lambdaMediaAds, у которого высокая конверсия и высокое удержание.

<div class="alert alert-success">

<b>Комментарий ревьюера v.2</b>

  ✔️ Каналы у нас не пересекаются.
</div>

### Выводы



Для решения задач, поставленных в начале исследования, был произведён анализ профилей пользователей приложения Procrastinate Pro+, а также были определены и исследованы следующие метрики:  LTV, ROI, САС, conversion rate, retention rate.

Анализ профилей пользователей позволил выявить следующие важные особенности:

- Пользователи приложения приходят из США, Германии, Великобритании и Франции. Количество новых пользователей из США значительно превышает количество пользователей из других стран - 100002. Из Великобритении и Франции пришло в приложение примерно одинаковое количество пользователей - 17575 и 17450 соответственно. Меньше всего пользователей пришло из Германии - 14981. Самая высокая доля платящих пользователей из США - 6.9%. Далее идут Германия (4.11%), Великобритания (3.98%) и Франция(3.8%).

- Большая часть пользователей для входа в приложение использует iPhone - 54479. Второе по популярности устройство - Android (35032). Примерно одинаковое количество пользователей использует для входа в приложение PC и Mac (30455 и 30042 соответственно). Наиболее высокие доли платящих у пользователей Apple: Mac - 6.4%, iPhone - 6.2%. Несколько ниже доля платящих у пользователей Android (5.9%). Самый низкий показатель у пользователей PC - 5%.

- Хотя количество органических пользователей самое значительное, доля платящих среди них самая низкая (2.1%). Самая высокая доля платящих приходит через FaceBoom (12.2%). Также высокая доля платящих приходит через AdNonSense (11.34%), lambdaMediaAds (10.47%) и TipTop (9.6%).

Анализ расходов на рекламу позволил сделать следующие наблюдения:

- больше всего средств было потрачено на рекламу в TipTop и Faceboom;
- при этом в мае траты на рекламу в Faceboom были несколько выше, но примерно с 21 недели (с июня) TipTop вышел в лидеры. Вклад в рекламу на этом ресурсе стабильно рос до 9 месяца с заметным пиком на 39 неделе;
- самыми дорогими по привлечению пользователями оказались пользователи, пришедшие через TipTop, FaceBoom и AdNonSense.

При оценке окупаемости рекламы были сделаны следующие выводы:
- реклама в целом не окупается. ROI к концу второй недели достигает примерно 80%;
 - САС показывает значительный рост начиная с серидины мая, рекламный бюджет заметно увеличивался на протяжении всего рассматриваемого периода;
- динамика ROI на 14 день показывает стабильную тенденцию к снижению.

Изучение окупаемости рекламы с разбивкой по устройствам показало:
- самыми дорогими для привлечения пользователями являются пользователи iPhone и Mac, а самыми недорогими - пользователи PC;
- между тем, окупается реклама ближе к концу второй недели только у пользователей PC;
- хотя конверсия пользователей iPhone и Mac самая высокая, а конверсия пользователей PC - самая низкая, удержание платящих пользователей PC стабильно самое высокое, а удержание пользователей iPhone и Mac - самое низкое.

Изучение окупаемости рекламы с разбивкой по странам показало:
- реклама в США не окупается совсем;
- расходы на привлечение пользователей из США были увеличены колосальным образом, начиная с середины мая, а рекламный бюджет по другим странам с конца мая наоборот снизился и остался стабильным до конца октября;
- лучшая окупаемость рекламы в Великобритании - начиная примерно с 5 дня, а в Германии и Франции - с 6;
- удержание платящих пользователей из США самое низкое, несмотря на самую высокую конверсию.

Изучение окупаемости рекламы с разбивкой по рекламным каналам показало:
- - самый высокий рекламный бюджет у TipTop, Faceboom и AdNonSense, но реклама на всех этих каналах не окупается;
- вложения в рекламу в TipTop, начиная с конца мая, сильно увеличивались;
- САС по другим источникам привлечения достаточно стабилен на протяжении всего периода;
- самая лучшая окупаемость у YRabbit.
- хотя конверсия в TipTop и AdNonSense самая высокая, удержание пользователей на данных ресурсах самое низкое.

В результате проведённого анализа показателей LTV, ROI, САС, conversion rate, retention rate выявились проблемные моменты в проведённых рекламных кампаниях. 
- В первую очередь обращает на себя внимание тот факт, что на протяжении всего рассматриваемого периода были значительно увеличены расходы на привлечение новых пользователей из США. К концу октября стоимость привлечения пользователя из США более чем в 4 раза превышала САС пользователей из других стран. Между тем, затраты на пользователей из США не окупились. Анализ удержания показал, что удержание пользователей из США самое низкое.
- Самыми дорогими для привлечения пользователями оказались пользователи, использующие iPhone и Mac. Однако, окупаются только пользователи, использующие PC. Дополнительное изучение удержания показало, что удержание пользователей iPhone и Mac - самое низкое.
- Значительным образом были увелиены расходу на рекламу через TipTop, однако, реклама на данном ресурсе не окупилась. Дополнительное изучение удержания показало, что удержание пользователей на данном ресурсе самое низкое.

*Рекомендации*:
- Провести дополнительное исследование возможных причин стабильно низкого удержания пользователей iPhone и Mac. Возможно, причина кроется в технических ошибках, не позволяющих пользователям заврешить своё посещение покупкой.

- Пересмотреть решение о высоких вложениях в рекламу на TipTop. Возможно, что скорректированный уровень рекламных вложений в данный ресурс позволит получить более эффективный результат, так как удержание на TipTop довольно высокое. А вот удержание на Faceboom и AdNonSense самое низкое, реклама на данных каналах также не окупается. Возможно, стоит пересмотреть решение о размещении рекламы на данных ресурсах. Стоит обратить внимание на каналы со стабильно высокой конверсией и высоким удержанием lambdaMediaAds и RocketSuperAds.

- Пересмотреть рекламную политику в отношении пользователей из США. Дополнительный анализ рекламных кампаний в США показал, что большинство пользователей пришли в приложение через каналы, реклама в которых не окупилась - FaceBoom и TipTop. Неуспех FaceBoom связан с очень низким удержанием, а TipTop - с завышенными тратами. По этим причинам реклама в США так и не окупилась. Необходимо скорректировать рекламный бюджет для TipTop, так как в целом на этом канале высокие и конверсия и удержание. Также стоит обратить внимание на рекламу черезе RocketSuperAds, где конверсия также довольно высокая и стабильно высокое удержание.

- Дополнительный анализ рекламы в Европе показал, что в единственным каналом, который не окупается, является AdNonSense. У этого канала самое низкое удержание. Перспективным для будущих рекламных кампаний может быть lambdaMediaAds, у которого высокая конверсия и высокое удержание.

<div class="alert alert-success">
  
  <b>Комментарий ревьюера</b>
    
  ✔️ Хорошие общие выводы!
</div>
<div class="alert alert-danger">
    
  <b>Комментарий ревьюера</b>
    
  ❌ Посмотри на конверсию у канала YRabbit, разве стоит советовать увеличение бюджета на этот канал? Мы, скорее всего, получим очередной TipTop. Нам надо на графиках найти окупающиеся каналы обязательно с хорошей конверсией и удержанием, в идеале с привязкой к регионам (США и Европа). У нас точно окупается больше одного канала. Маркетологи оценят! 🙂
</div>


<div class="alert alert-block alert-info">
  
  <b>Комментарий cтудента</b>
    
  Сергей, подскажи, пожалуйста, по какой причине на графиках динамики удержания присутствуют прерывания? Это говорит об отсутствии данных об удержании на 14 день по определённым каналам для платящих? Или этому есть ещё какие-то объяснения?
</div>
<div class="alert alert-success">
  
  <b>Комментарий ревьюера v.2</b>
    
  ✔️ Вывод преобразился, теперь видно не только в чем проблемы, но и даны рекомендации для их устранений.  👍
    
    
У нас по Европе и части каналов очень мало пользователей (видели при расчете доли платящих), просто в какой-то момент за 2 недели не было не одного покупателя, когорта получилась пустая - отсюда и разрывы на графиках.
</div>

<div style="border:solid black 2px; padding: 20px">
     
  <b>Чек-лист доработок</b>
    
  
  📃 Написать промежуточные выводы по шагам проекта.
    
  📃 Пересмотреть альтернативные рекомендации отделу маркетинга.
    
Жду твой проект на повторное ревью. Не стесняйся оставлять комментарии и задавать вопросы. Удачи! Все получится.
</div>